In [1]:
#Necessary imports for plotting, pandas, math operations, and multiple print statements.
import pandas as pd
from scipy.stats import chi2, chi2_contingency
from IPython.display import Image
import matplotlib.pyplot as plt
from math import pi
import numpy as np
import seaborn as sns
%matplotlib inline
import re
import enchant

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

ImportError: The 'enchant' C library was not found and maybe needs to be installed.
See  https://pyenchant.github.io/pyenchant/install.html
for details


In [ ]:
# def removeDuplicates(scripts_basics_df):
#     # Key = IMDB ID , Value = Size of script
#     smallestMovieScriptMap = {}
#     # Key = IMDB ID , Value = index
#     keepIndexMap = {}

#     #show all columns
#     scripts_basics_df.dtypes

#     scripts_basics_df['Scripts'] = scripts_basics_df['Scripts'].apply(str)

#     for index, row in scripts_basics_df.iterrows():
#         constID = row['IMDB ID']
#         currentSize = len(row['Scripts'])

#         if constID in smallestMovieScriptMap:
#             smallestSize = smallestMovieScriptMap[constID]
#             if currentSize < smallestSize:
#                 smallestMovieScriptMap[constID] = currentSize
#                 keepIndexMap[constID] = index
#         else:
#             smallestMovieScriptMap[constID] = currentSize
#             keepIndexMap[constID] = index
#     # keep this removed, this removes first instance
#     # scripts_basics_df = scripts_basics_df[~scripts_basics_df.index.duplicated(keep='first')]
#     return scripts_basics_df[scripts_basics_df.index.isin(keepIndexMap.values())]


In [ ]:
# #Read basics.tsv
# basics_df = pd.read_csv("datasets/basics.tsv", sep="\t")

# #Filter basics_df
# basics_df = basics_df[basics_df["titleType"] == "movie"]
# basics_df = basics_df[basics_df["genres"] != "\\N"]
# # basics_df = basics_df[basics_df["startYear"] != "\\N"]
# # basics_df["startYear"] = basics_df["startYear"].apply(int)
# # basics_df = basics_df[basics_df["startYear"] >= 1960]
# basics_df = basics_df.set_index("tconst")

# #Filter akas_df
# akas_df = pd.read_csv("datasets/akas.tsv", sep="\t", dtype={
#     "titleId": "string", 
#     "ordering": int, 
#     "title": "string", 
#     "region":"string",
#     "language":"string"
# })

# akas_df = akas_df[akas_df["region"].isin(["US","CA"])]
# akas_df = akas_df[akas_df["language"] == "en"]
# akas_df = akas_df.set_index("titleId")

# #Join
# akas_basics_df = basics_df.join(akas_df, how='inner')

In [ ]:
# #Read scripts.csv
# scripts_df = pd.read_csv("datasets/scripts.csv", dtype={
#     "IMDB ID": "string", 
#     "Scripts": "string", 
#     "release year": int, 
# });
# scripts_df = scripts_df.set_index("IMDB ID");
# scripts_df = scripts_df.drop(['genres'], axis=1)

# scripts_basics_df = akas_basics_df.join(scripts_df, how='inner')
# scripts_basics_df=scripts_basics_df.reset_index()
# scripts_basics_df = scripts_basics_df.rename(columns={'index': 'IMDB ID'})
# scripts_basics_df = removeDuplicates(scripts_basics_df)

# printing one script
# pd.set_option('display.max_colwidth', None)
# scripts_basics_df.loc[scripts_basics_df['IMDB ID'] == "tt0107290"]["Scripts"]

# only need this now since we have cleaned data 
scripts_basics_df = pd.read_csv("datasets/clean.csv")

# cleaning data
scripts_basics_df = scripts_basics_df.dropna() #delete NaN values
scripts_basics_df = scripts_basics_df.astype({'Scripts':'string'})
scripts_basics_df["Scripts"] = scripts_basics_df["Scripts"].replace(to_replace=r"[^a-zA-Z0-9\ ]", value="", regex=True)
# scripts_basics_df["Scripts"] = scripts_basics_df["Scripts"].replace(to_replace=r"<.*>", value="", regex=True)
# scripts_basics_df["Scripts"] = scripts_basics_df["Scripts"].replace(to_replace=r"\ .*@.*\ ", value=" ", regex=True)

# splitting genres
scripts_basics_df["genres"] = scripts_basics_df["genres"].str.split(",")
scripts_basics_df = scripts_basics_df.explode("genres")

# convert "genres" from categorical to quantitative
scripts_basics_df["genres_num"] = scripts_basics_df["genres"].map({'Comedy': 0, 'Fantasy': 1, 'Romance': 2, 'Drama': 3, 
                                                                   'Thriller': 4, 'Action': 5, 'Crime': 6, 'Adventure': 7, 
                                                                   'Family': 8, 'Mystery': 9, 'Biography': 10, 'Music': 11,
                                                                   'Horror': 12, 'Sci-Fi': 13, 'Animation': 14,  'Sport': 15, 
                                                                   'History': 16,  'Western': 17, 'War': 18,  'Musical': 19, 
                                                                   'Documentary': 20
                                                                   })

display(scripts_basics_df[["originalTitle", "Scripts", "genres", "genres_num"]])
# display(scripts_basics_df["genres"].unique())
# display(scripts_basics_df["genres"].unique().size)
# display(scripts_basics_df["genres"].value_counts())

In [ ]:
# tf idf

from sklearn.feature_extraction.text import TfidfVectorizer

toy_df = scripts_basics_df[["originalTitle", "Scripts", "genres", "genres_num"]].loc[:100].copy()
toy_df = toy_df[toy_df["Scripts"].str.contains("<NA>") == False]
display(toy_df)

tfidf = TfidfVectorizer()
response = tfidf.fit_transform(toy_df["Scripts"])
feature_names = tfidf.get_feature_names()
vectorized_df = pd.DataFrame(response.toarray()).set_axis([feature_names], axis=1).set_axis(toy_df["originalTitle"], axis=0)
display(vectorized_df)

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from numpy import dot
from numpy.linalg import norm

X_train = vectorized_df.iloc[:150].copy()
X_test = vectorized_df.iloc[151:].copy()
y_train = toy_df["genres_num"].iloc[:150]
y_test = toy_df["genres_num"].iloc[151:]

# X_train
# X_test
# y_train
# y_test

def cos_sim(x, y):
    # print(dot(x, y) / (norm(x) * norm(y)))
    return dot(x, y) / (norm(x) * norm(y))

knn_model = KNeighborsRegressor(n_neighbors=15, metric=cos_sim)
# knn_model = KNeighborsRegressor(n_neighbors=15)
knn_model.fit(X=X_train, y=y_train)

In [ ]:
pd.set_option('display.max_colwidth', None)
# evaluate the performance of our model using MSE
# - "predict" y values of X_train values
# - compare these predicted values to the actual values y_train to evaluate
from sklearn.metrics import mean_squared_error
train_preds = knn_model.predict(X_train)
y_train.to_numpy()[0:5]
train_preds[0:5]
mse = mean_squared_error(y_train, train_preds)
mse

# compare predicted values on the testing set to the actual values y_test
test_preds = knn_model.predict(X_test)
mse = mean_squared_error(y_test, test_preds)
mse